In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import keras
from keras.callbacks import Callback
from keras.layers import Input, Conv2D, Conv2DTranspose, Add, Activation
from keras.models import Model
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import math
from torch.utils.data import Dataset, DataLoader, TensorDataset
from pathlib import Path
import torch
from tqdm import tqdm
from collections import defaultdict

print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

In [ ]:
# you need to add the folder to your drive
# I added Davids folder as a shortcut to my folder /cvue23

base = Path.cwd() / 'drive' / 'MyDrive' / 'cvue23' / 'preprocessed_data'

In [ ]:
map_label_to_name = ['no_person', 'idle','sitting', 'laying']

class DataGenerator(keras.utils.Sequence):
    def __init__(self, basedir: Path, batch_size: int, filter_no_person: bool, filter_no_trees: bool):
        if not basedir.exists():
            ValueError('Datafolder does not exist. Add it to your drive and try again. Maybe restart the runtime.')

        self.basedir = basedir
        self.batch_size = batch_size
        self.filter_no_person = filter_no_person
        self.filter_no_trees  = filter_no_trees
        self.filenames = self.__filter()

    def __filter(self):

        files = []
        self.pose_distribution = defaultdict(int)
        self.trees_distribution = defaultdict(int)

        for i, path in tqdm(enumerate(self.basedir.iterdir()), total=27280):

            loaded = np.load(path)
            pose, trees = loaded['pose'], loaded['trees']

            self.pose_distribution[pose.item()] += 1
            self.trees_distribution[trees.item()] += 1

            fname = path.name
            if self.filter_no_person and pose == 0:
                continue
            elif self.filter_no_trees and trees == 0:
                continue
            else:
                files.append(fname)

        return files

    def load(self, path):
        loaded = np.load(path)
        x = loaded['x'] / 255
        y = loaded['y'] / 255
        return x, y

    def __len__(self):
        return math.ceil(len(self.filenames) / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        high = min(low + self.batch_size, len(self.filenames))

        X, Y = [],[]
        for fname in self.filenames[low:high]:
            x,y = self.load(self.basedir / fname)
            X.append(x)
            Y.append(y)

        return np.stack(X), np.stack(Y)

    def print_distribution(self):
        print(f'Pose distribution (filtered={self.filter_no_person})')
        for key, value in self.pose_distribution.items():
            print(f'{map_label_to_name[key]} - {value}')

        print(f'Trees distribution (filtered={self.filter_no_trees})')
        for key, value in self.trees_distribution.items():
            print(f'{key} trees per ha - {value}')

datagen = DataGenerator(
    basedir=base / 'training_set',
    batch_size=16,
    filter_no_person=True,
    filter_no_trees=False
)

In [ ]:
# you can set the batchsize later, no need to reload and refilter
datagen.batch_size=16
print(len(datagen))

# changing batch_size changes length
datagen.batch_size=1
print(len(datagen))

# check out the distribution of the samples
datagen.print_distribution()

# load a sample batch and check the shape
datagen.batch_size=16
_x, _y = datagen[0]
print(_x.shape, _y.shape)

In [ ]:
# for debugging
def plot_image_grid(images_array, grid_width=10, grid_height=10):

    if images_array.shape[0] != grid_width * grid_height:
        raise ValueError("The number of images does not match the grid size.")

    fig, axes = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))

    for i, ax in enumerate(axes.flatten()):
        ax.imshow(images_array[i], cmap='gray', interpolation='none')
        ax.axis('off')

    plt.tight_layout()
    plt.show()


def encoder(x, num_features, num_layers, residual_every=2):
    x = Conv2D(num_features, kernel_size=3, strides=2, padding='same', activation='relu')(x)

    # Save the output of conv layers at even indices
    residuals = []

    # Encoder
    for i in range(num_layers - 1):
        x = Conv2D(num_features, kernel_size=3, padding='same', activation='relu')(x)
        if (i + 1) % residual_every == 0:
            residuals.append(x)

    return x, residuals

def decoder(x, num_features, num_layers, residuals, residual_every=2):

    # Decoder
    for i in range(num_layers - 1):
        x = Conv2DTranspose(num_features, kernel_size=3, padding='same')(x)

        if (i + 1 + num_layers) % residual_every == 0 and residuals:
            res = residuals.pop()
            x = Add()([x, res])

        x = Activation('relu')(x)

    if residuals: raise ValueError('There are unused residual connections')

    # create 1-channel output
    x = Conv2DTranspose(1, kernel_size=3, strides=2, padding='same')(x)

    return x

def REDNet(num_layers, num_features, channel_size):
    '''Model definition with keras functional layers api'''

    inputs = Input(shape=(None, None, channel_size))

    x, residuals = encoder(inputs, num_features, num_layers)

    x = decoder(x, num_features, num_layers, residuals)

    # Add input residual, needed to do 1x1 conv to adapt channels
    residual = Conv2DTranspose(1, kernel_size=1, padding='same')(inputs)
    outputs = Add()([x, residual])
    outputs = Activation('relu')(outputs)

    # Create model
    model = Model(inputs=inputs, outputs=outputs, name=f'REDNet{num_layers*2}')
    return model

class PredictionCallback(Callback):
    def __init__(self, interval, x_val, y_val):
        super(PredictionCallback, self).__init__()
        self.interval = interval
        self.x_val = x_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            preds = self.model.predict(self.x_val).squeeze()
            plot_image_grid(np.concatenate([self.x_val[:,:,:,1], preds, self.y_val]), preds.shape[0], 3)

In [ ]:
def load_data(path):
  # load the mini dataset
  loaded = np.load(path)
  x, y, labels = loaded['x'], loaded['y'], loaded['labels']

  # normalize dataset
  assert y.max() == 255
  assert x.max() == 255

  y = y / 255
  x = x / 255
  return x, y, labels

x_val, y_val, _ = load_data(base / 'part_1_original_test_100.npz')

In [ ]:
# compile the model
model = REDNet(
    num_layers=9,
    num_features=64,
    channel_size=x_val.shape[-1]
)

opt = keras.optimizers.Adam(
    learning_rate=0.00001
)

loss = keras.losses.MeanSquaredError(
    reduction="sum_over_batch_size",
    name="mse"
)

prediction_callback = PredictionCallback(interval=2, x_val=x_val[:8], y_val=y_val[:8])
model.compile(loss=loss,optimizer=opt)

In [ ]:
# train on the dataset
history = model.fit(
    x=datagen,
    epochs=20,
    validation_data=(x_val, y_val),
    callbacks=[prediction_callback],
    shuffle=True,
)
# Save the weights
model.save_weights(base.parent / 'models' / 'new_2')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('MSE Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()